In [2]:
!pip install ipywidgets

from IPython.display import display, clear_output
!pip install --upgrade notebook ipywidgets

import ipywidgets as widgets
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
widgets.IntSlider()
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import joblib
from sklearn.preprocessing import StandardScaler


In [14]:

# Modelo
try:
    best_model = joblib.load("Linear_Regression_model.pkl")
    model_exists = True
    print("Best model loaded successfully")
except:
    best_model = None
    model_exists = False
    print("⚠️Best model not found. Using demo calculation.")

# Features exactos usados en tu modelo
features = [
    'Temperature', 'Humidity', 'SquareFootage', 'Occupancy',
    'HVACUsage', 'LightingUsage', 'DayOfWeek', 'Holiday', 'Hour', 'Day', 'Month',
    'WeekendLabel', 'TimePeriodLabel'
]

# Escalador (ajusta si tienes scaler original guardado)
scaler = joblib.load("scaler.pkl")

# Mapas
day_mapping = {
    'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4,
    'Friday': 5, 'Saturday': 6, 'Sunday': 7
}
holiday_mapping = {'No': 0, 'Yes': 1}
hvac_mapping = {'Off': 0, 'On': 1}
lighting_mapping = {'Off': 0, 'On': 1}
weekend_mapping = {'Weekday': 0, 'Weekend': 1}
timeperiod_mapping = {'Afternoon': 0, 'Evening': 1, 'Morning': 2, 'Night': 3}

# Widgets
day_of_week = widgets.Dropdown(
    options=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    description="Day:"
)
holiday = widgets.Dropdown(options=['No', 'Yes'], description="Holiday:")
hour = widgets.IntSlider(value=12, min=0, max=23, step=1, description="Hour:")
day = widgets.IntSlider(value=15, min=1, max=31, step=1, description="Day:")
month = widgets.IntSlider(value=6, min=1, max=12, step=1, description="Month:")
temperature = widgets.FloatSlider(value=22.0, min=10, max=35, step=0.5, description="Temp (°C):")
humidity = widgets.FloatSlider(value=50.0, min=20, max=80, step=1.0, description="Humidity (%):")
square_footage = widgets.IntSlider(value=1500, min=500, max=5000, step=100, description="Area (ft²):")
occupancy = widgets.IntSlider(value=5, min=1, max=20, step=1, description="Occupancy:")
hvac_usage = widgets.Dropdown(options=['Off', 'On'], description="HVAC:")
lighting_usage = widgets.Dropdown(options=['Off', 'On'], description="Lighting:")

button = widgets.Button(description=" Predict Energy", button_style="success")
output = widgets.Output()

def get_time_period(hour):
    if 4 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

def predict_callback(b):
    with output:
        clear_output()
        # Calcula Weekday/Weekend
        day_num = day_mapping[day_of_week.value]
        weekend_label = 'Weekend' if day_num >= 6 else 'Weekday'
        # Calcula TimePeriodLabel
        tp_label = get_time_period(hour.value)
        # Prepara input_dict
        input_dict = {
            'Temperature': temperature.value,
            'Humidity': humidity.value,
            'SquareFootage': square_footage.value,
            'Occupancy': occupancy.value,
            'HVACUsage': hvac_mapping[hvac_usage.value],
            'LightingUsage': lighting_mapping[lighting_usage.value],
            'DayOfWeek': day_num,
            'Holiday': holiday_mapping[holiday.value],
            'Hour': hour.value,
            'Day': day.value,
            'Month': month.value,
            'WeekendLabel': weekend_mapping[weekend_label],
            'TimePeriodLabel': timeperiod_mapping[tp_label]
        }
        input_data = pd.DataFrame([input_dict])[features]

        if model_exists:
            input_scaled = scaler.transform(input_data)
            prediction = best_model.predict(input_scaled)[0]
            print(f"Predicted Energy Consumption: {prediction:.2f} kWh")
        else:
            base = square_footage.value * 0.04
            temp_factor = abs(temperature.value - 22) * 2
            hvac_factor = 20 if hvac_usage.value == 'On' else 0
            lighting_factor = 8 if lighting_usage.value == 'On' else 0
            occupancy_factor = occupancy.value * 2
            prediction = base + temp_factor + hvac_factor + lighting_factor + occupancy_factor
            print(f"Energy Consumption: {prediction:.2f} kWh (demo)")

button.on_click(predict_callback)

# Display interface
display(widgets.VBox([
    widgets.HBox([day_of_week, holiday]),
    widgets.HBox([hour, day, month]),
    temperature, humidity, square_footage, occupancy,
    widgets.HBox([hvac_usage, lighting_usage]),
    button, output
]))

Best model loaded successfully


In [ ]:
input_example = {
    'Temperature': 25.0,
    'Humidity': 45.0,
    'SquareFootage': 1800,
    'Occupancy': 8,
    'HVACUsage': 1,       # 1 = On, 0 = Off
    'LightingUsage': 1,   # 1 = On, 0 = Off
    'DayOfWeek': 6,       # 6 = Saturday
    'Holiday': 0,         # 1 = Yes, 0 = No
    'Hour': 14,
    'Day': 20,
    'Month': 7
}